# Credentials

__Put Testnet or Prod credentials here__:
(Testnet here: https://testnet.binancefuture.com/en/futures/BTCUSDT)

Testnet resets each day, but your credentials wont change.

In [1]:
api_key = "6ce63f3406fd8ebbff01054a66c25fe3c851c45932088c8ca3131a7005188462"
secret_key = "aa3ea32929252467fa5ffeac5818c95beabfb5dba691ef445e7eaa31ea0d15f6"

# Python Api Wrapper

It is important to install python wrappers of binance API (one is not made by Binance, use at your own risk), just run on terminal:

__pip install python-binance__

__pip install binance-futures-connector__

Currently unofficial wrapper has problems retrieving real-time prices (problem with websockets), so we use the official Api wrapper of Binance to this task.

Useful links:

__Binance Official Api Docs__: https://binance-docs.github.io/apidocs/futures/en/#account-trades-endpoints

__Unofficial Python Api Wrapper__: https://python-binance.readthedocs.io/en/latest/overview.html

__Official Binance Api Wrapper__: https://github.com/binance/binance-futures-connector-python 

Api Wrapper sometimes its not specific on arguments for functions, so it´s useful to check that args on official Api docs

__This is the code for live streaming that is going to be replaced with official api__

In [5]:
from binance import ThreadedWebsocketManager
import time
twm = ThreadedWebsocketManager(testnet = True)
twm.start()
twm.start_kline_futures_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")
time.sleep(10)
twm.stop()

'btcusdt_perpetual@continuousKline_1m'

CANCEL read_loop
Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ThreadedApiManager.start_listener() done, defined at /home/mauricio/anaconda3/lib/python3.9/site-packages/binance/threaded_stream.py:42> exception=RuntimeError("Task <Task pending name='Task-14' coro=<Queue.get() running at /home/mauricio/anaconda3/lib/python3.9/asyncio/queues.py:166> cb=[_release_waiter(<Future pendi...112a0c370>()]>)() at /home/mauricio/anaconda3/lib/python3.9/asyncio/tasks.py:416]> got Future <Future pending> attached to a different loop")>
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/threaded_stream.py", line 46, in start_listener
    msg = await asyncio.wait_for(s.recv(), 3)
  File "/home/mauricio/anaconda3/lib/python3.9/asyncio/tasks.py", line 479, in wait_for
    return fut.result()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/streams.py", line 197, in recv
    res = await asyncio.wait_f

__Useful code for streaming__

In [1]:
import time
#import logging
#from binance.lib.utils import config_logging
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient

#config_logging(logging, logging.DEBUG)


def message_handler(message):
    print(message)


my_client = CMFuturesWebsocketClient()
my_client.start()

my_client.kline(
    symbol="btcusd_perp",
    id=12,
    interval="1m",
    callback=message_handler,
)

time.sleep(10)

#logging.debug("closing ws connection")
my_client.stop()

{'id': 12, 'result': None}
{'e': 'kline', 'E': 1660196910471, 's': 'BTCUSD_PERP', 'k': {'t': 1660196880000, 'T': 1660196939999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503634938, 'L': 503635180, 'o': '24634.7', 'c': '24633.5', 'h': '24636.7', 'l': '24627.9', 'v': '11179', 'n': 243, 'x': False, 'q': '45.38384832', 'V': '5765', 'Q': '23.40584244', 'B': '0'}}
{'e': 'kline', 'E': 1660196911325, 's': 'BTCUSD_PERP', 'k': {'t': 1660196880000, 'T': 1660196939999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503634938, 'L': 503635181, 'o': '24634.7', 'c': '24633.5', 'h': '24636.7', 'l': '24627.9', 'v': '11180', 'n': 244, 'x': False, 'q': '45.38790783', 'V': '5766', 'Q': '23.40990195', 'B': '0'}}
{'e': 'kline', 'E': 1660196911577, 's': 'BTCUSD_PERP', 'k': {'t': 1660196880000, 'T': 1660196939999, 's': 'BTCUSD_PERP', 'i': '1m', 'f': 503634938, 'L': 503635240, 'o': '24634.7', 'c': '24637.9', 'h': '24637.9', 'l': '24627.9', 'v': '13463', 'n': 303, 'x': False, 'q': '54.65514345', 'V': '8049', 'Q': '32.67713757', 

# Streaming data

This is a simple class for streaming candles. It adds bars to a dataframe on a specified time.

__Important__: You maybe need to restart kernel each time you close a connection.

In [1]:
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np

In [2]:
class FuturesTrader():
    def __init__(self, interval = "1m", symbol="btcusd"):
        self.stream = None
        self.data = pd.DataFrame(columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Complete"])
        self.data["Date"] = pd.to_datetime(self.data.iloc[:,0], unit = "ms")
        self.data.set_index("Date", inplace = True)
        self.interval = interval
        self.symbol = symbol
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
    def start_streaming(self):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=1,
            interval=self.interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()

In [3]:
trader = FuturesTrader(interval="1m", symbol="btcusd")

In [4]:
trader.start_streaming()

.

In [5]:
trader.stop_streaming()

In [6]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-08-11 21:59:00,24183.7,24183.7,24183.7,24183.7,818.0,False


# Retrieving historical data

When running the bot trader, we need the latest historical data to process all the needed metrics in real time. This method retrieves the latest __n__ candlesticks data on the required interval.

This lines are useful for saving candlesticks info for Neural Networks

api info:  https://binance-docs.github.io/apidocs/futures/en/#kline-candlestick-data

api wrapper method info: https://python-binance.readthedocs.io/en/latest/market_data.html#id7

__Important__: 

1. The used __function "futures_historical_klines" doesn't behave according the restrictions of binance api__, for example, the limit is not 1500 records as binance says. For example, if you want more than 1500 candlesticks, i think it makes more than one request and prepares the result for you...

2. Also, notice that __the code shows all the info retrieved by Binance API__, if you want one that is not used, add it to section "Use this info".

3. Notice that __the last record retrieved has not completed his time period__, so its not accurate info.

In [4]:
from binance.client import Client
from datetime import datetime, timedelta
import pandas as pd

#### VARIABLES ####
max_number_of_candles = 5 
interval_of_candles = "1m"
symbol = "btcusdt"
#### Available intervals for candles ####
time_to_past = {
 "1m"  : timedelta(minutes=max_number_of_candles),
 "3m"  : timedelta(minutes=max_number_of_candles*3),
 "5m"  : timedelta(minutes=max_number_of_candles*5),
 "15m" : timedelta(minutes=max_number_of_candles*15),
 "30m" : timedelta(minutes=max_number_of_candles*30),
 "1h"  : timedelta(hours=max_number_of_candles),
 "2h"  : timedelta(hours=max_number_of_candles*2),
 "4h"  : timedelta(hours=max_number_of_candles*4),
 "6h"  : timedelta(hours=max_number_of_candles*6),
 "8h"  : timedelta(hours=max_number_of_candles*8),
 "12h" : timedelta(hours=max_number_of_candles*12),
 "1d"  : timedelta(days=max_number_of_candles),
 "3d"  : timedelta(days=max_number_of_candles*3),
 "1w"  : timedelta(days=max_number_of_candles*7),
 "1M"  : timedelta(days=max_number_of_candles*28) #this may give less than the desired candles because each month has different amount of days
}
#### Request info ####

client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True) # Testnet!!!

now = datetime.utcnow()
past = str(now - time_to_past[interval_of_candles])

bars = client.futures_historical_klines(symbol = symbol, 
                                        interval = interval_of_candles, 
                                        start_str =past,
                                        end_str = None)
#### Prepare dataframe ####
df = pd.DataFrame(bars)
df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
#### Info given by Binance ####
df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
              "Close Time", "Quote Asset Volume", "Number of Trades",
              "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
#### Use this info ####
use_columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
######################
df = df[use_columns].copy()
df.set_index("Date", inplace = True)
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors = "coerce")
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-11 22:05:00,24158.4,24175.4,24158.4,24168.4,68.066
2022-08-11 22:06:00,24161.0,24169.6,24091.2,24091.2,87.734
2022-08-11 22:07:00,24129.8,24141.3,24117.3,24117.3,53.005
2022-08-11 22:08:00,24117.3,24154.7,24117.3,24117.3,171.187
2022-08-11 22:09:00,24140.8,24157.1,24132.5,24157.1,38.092


Then we can save the data to prevent requesting too many times from futures api

In [42]:
df.to_csv("out.csv")

You can load the data running:

In [63]:
loaded = pd.read_csv('out.csv')
loaded.set_index("Date", inplace = True)
loaded

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-04 05:50:00,23780.0,23780.0,23198.0,23198.0,84.921
2022-08-04 05:51:00,23228.2,23900.0,23198.0,23900.0,84.941
2022-08-04 05:52:00,23852.2,23898.0,23198.0,23898.0,132.174
2022-08-04 05:53:00,23898.0,23900.0,23198.0,23228.2,74.257
2022-08-04 05:54:00,23520.4,23946.0,23198.0,23653.4,83.717
...,...,...,...,...,...
2022-08-11 04:26:00,24344.5,24344.5,24200.0,24317.6,75.895
2022-08-11 04:27:00,24312.2,24317.6,24150.0,24308.0,99.008
2022-08-11 04:28:00,24310.1,24313.6,24305.6,24311.0,21.804


# Combining historical and streaming data in a class

This is a class that __keeps historical data and updates it with streaming info__. It is useful for our trading bot to have accurate metrics when first loading.

In [2]:
from binance.client import Client
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
class FuturesTrader():
    def __init__(self, symbol="btcusd", testnet = True):
        ####API CONNECTIONS ####
        self.stream = None
        self.client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = testnet)
        #######################
        self.data = None
        self.symbol = symbol 
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            print("candle completed", end="")
            #execute trading functions here!!
        
    def start_streaming(self, interval="1m"):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=1,
            interval=interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()
        
    def get_most_recent_data(self, num_candles=100, interval = "1m"):
        #### Get start time for candles ####
        now = datetime.utcnow()
        past = str(now - self.available_intervals(num_candles)[interval])
        #### "BTCUSD" is not valid with this method ####
        symbol = self.symbol + "t" if self.symbol.endswith('usd') else self.symbol
        #### Request candles and prepare the df ####
        bars = self.client.futures_historical_klines(symbol = symbol, 
                                        interval = interval, 
                                        start_str =past,
                                        end_str = None)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Close Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]    
        self.data = df
    
    def start_trading(self, num_candles = 100, interval = "1m"):
        if interval in self.available_intervals(num_candles).keys():
            self.get_most_recent_data(num_candles = num_candles, interval=interval)
            self.start_streaming(interval)
        else:
            print("That interval is not available")
            
    def stop_trading(self):
        self.stop_streaming()
        #print ending metrics here!!
        
    def available_intervals(self, candles_required):
        '''
        Helper function for "get_most_recent_data" method.
        
        '''
        return {
            "1m"  : timedelta(minutes=candles_required),
            "3m"  : timedelta(minutes=candles_required*3),
            "5m"  : timedelta(minutes=candles_required*5),
            "15m" : timedelta(minutes=candles_required*15),
            "30m" : timedelta(minutes=candles_required*30),
            "1h"  : timedelta(hours=candles_required),
            "2h"  : timedelta(hours=candles_required*2),
            "4h"  : timedelta(hours=candles_required*4),
            "6h"  : timedelta(hours=candles_required*6),
            "8h"  : timedelta(hours=candles_required*8),
            "12h" : timedelta(hours=candles_required*12),
            "1d"  : timedelta(days=candles_required),
            "3d"  : timedelta(days=candles_required*3),
            "1w"  : timedelta(days=candles_required*7),
            "1M"  : timedelta(days=candles_required*28) #this may give less than the desired candles because each month has different amount of days
        }     

In [4]:
trader = FuturesTrader(symbol="btcusd", testnet = True)

In [5]:
trader.start_trading(interval="1m", num_candles=3)

......

In [6]:
trader.stop_trading()

In [7]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-08-11 23:33:00,24224.2,24229.4,23905.7,23910.1,25.989,True
2022-08-11 23:34:00,23910.1,24615.0,23887.5,23894.8,360.989,True
2022-08-11 23:35:00,23891.5,23891.5,23869.8,23869.9,18675.000,True
2022-08-11 23:36:00,23869.8,23880.2,23869.8,23880.2,10447.000,False
